### Import all necessay Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf
from tensorflow.keras import Sequential,Model
from tensorflow.python.keras import regularizers
from tensorflow.keras.layers import Dense,Dropout,Activation,MaxPooling2D,Flatten,Conv2D,BatchNormalization,GlobalAveragePooling2D
from tensorflow import keras
from keras.applications.vgg16 import VGG16, preprocess_input
from skimage.transform import resize
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler

### Read the Dataset

In [14]:
cifar10 = tf.keras.datasets.cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

### Check the Dimensions of Dataset

In [15]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

### Check CLasses of Dataset

In [16]:
print(np.unique(y_train))
print(np.unique(y_test))

[0 1 2 3 4 5 6 7 8 9]
[0 1 2 3 4 5 6 7 8 9]


In [8]:
y_train = y_train.reshape(-1,)
y_test = y_test.reshape(-1,)
print('Y Training shape: ',y_train.shape)
print('Y Testing shape: ',y_test.shape)

Y Training shape:  (50000,)
Y Testing shape:  (10000,)


In [10]:
unique, counts = np.unique(y_train, return_counts=True)
print("Train labels: ", dict(zip(unique, counts)))

unique, counts = np.unique(y_test, return_counts=True)
print("\nTest labels: ", dict(zip(unique, counts)))

Train labels:  {0: 5000, 1: 5000, 2: 5000, 3: 5000, 4: 5000, 5: 5000, 6: 5000, 7: 5000, 8: 5000, 9: 5000}

Test labels:  {0: 1000, 1: 1000, 2: 1000, 3: 1000, 4: 1000, 5: 1000, 6: 1000, 7: 1000, 8: 1000, 9: 1000}


In [17]:
print(X_train.max())
print(X_train.min())
X_train = X_train/255.0
X_test = X_test/255.0
print(X_train.max())
print(X_train.min())

255
0
1.0
0.0


In [18]:
X_train.shape,y_test.shape

((50000, 32, 32, 3), (10000, 1))

In [19]:
no_classes = 10

In [21]:
base_model = VGG16(weights = "imagenet",input_shape=(32, 32, 3),include_top = False)

for layers in base_model.layers:
    layers.trainable = False


In [22]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [24]:
def vgg16_pretrained():    
    model = Sequential(
        [
            base_model,
            GlobalAveragePooling2D(),
            Flatten(),
            Dense(512, activation='relu'), 
            Dropout(0.4),
            Dense(256, activation='relu'),
            Dropout(0.4),
            Dense(10, activation='softmax')
        ]
    )
    return model

In [25]:
model = vgg16_pretrained()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                        

In [26]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

history = model.fit(X_train,y_train,epochs = 10,batch_size = 16,validation_data=(X_test,y_test))

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training Loss")
ax[0].plot(history.history['val_loss'], color='r', label="Validation Loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training Accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation Accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
plt.imshow(X_test[0])

In [ ]:
Y_pred = model.predict(x_test[0])
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert testing observations to one hot vectors
Y_true = np.argmax(y_test,axis = 1)
# compute the confusion matrix
confusion_mtx = tf.math.confusion_matrix(Y_true, Y_pred_classes) 

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(confusion_mtx, annot=True, fmt='g',cmap='gnuplot')

In [ ]:
from sklearn.metrics import confusion_matrix , classification_report
print("Classification Report: \n", classification_report(Y_true, Y_pred_classes))